In [1]:
import os
import requests
import pandas as pd
from datetime import datetime, timedelta
import json
from pathlib import Path
import time

In [2]:
BASE_URL = "https://api.the-odds-api.com/v4/historical"
API_KEY = os.environ.get("ODDS_API_KEY")
SPORT = "cricket_ipl"

In [3]:
def get_historical_odds(event_id: str, date: str):
    try:
        url = f"{BASE_URL}/sports/{SPORT}/events/{event_id}/odds"
        params = {
            "apiKey": API_KEY,
            "regions": "us",
            "markets": "h2h",
            "event_id": event_id,
            "bookmakers": "fanduel",
            "date": date
        }
        response = requests.get(url, params=params)
        response.raise_for_status()
        return response.json()
    
    except requests.exceptions.RequestException as e:
        print(e)
        return None

In [4]:
def get_timestamps_in_range(start_str: str, end_str: str, interval_minutes: int = 5) -> list:
    timestamps = []
    if not start_str or not end_str:
        return timestamps
    
    start_time = datetime.strptime(start_str, '%Y-%m-%dT%H:%M:%SZ')
    end_time = datetime.strptime(end_str, '%Y-%m-%dT%H:%M:%SZ')
    
    current = start_time
    while current <= end_time:
        timestamps.append(current.strftime('%Y-%m-%dT%H:%M:%SZ'))
        current += timedelta(minutes=interval_minutes)
    return timestamps

In [ ]:
def get_odds(row: pd.Series) -> None:
    the_odds_api_match_id = row['the_odds_api_id']
    output_dir = Path('odds')
    
    output_file = output_dir / f"{the_odds_api_match_id}.json"
    if output_file.exists():
        print(f"Skipping {row['sportmonks_id']} : {row['home_team']} vs {row['away_team']} - data exists")
        return
    
    if row['abandoned'] == 'TRUE':
        print(f"Skipping {row['sportmonks_id']} : {row['home_team']} vs {row['away_team']} - match abandoned")
        return
        
    timestamps = get_timestamps_in_range(row['start'], row['end'])
    if not timestamps:
        print(f"Skipping {row['sportmonks_id']} : {row['home_team']} vs {row['away_team']} - no valid timestamps")
        return
        
    odds_data = []
    for ts in timestamps:
        try:
            data = get_historical_odds(the_odds_api_match_id, ts)
            if data:
                odds_data.append({
                    'timestamp': ts,
                    'odds': data
                })
        except Exception as e:
            print(f"Error fetching odds for {the_odds_api_match_id} at {ts}: {str(e)}")
            continue
    
    if odds_data:
        output_file.parent.mkdir(parents=True, exist_ok=True)
        output_file.write_text(json.dumps(odds_data, indent=2))
        print(f"Saved odds for {row['home_team']} vs {row['away_team']}")
    else:
        print(f"No odds data collected for {row['home_team']} vs {row['away_team']}")

In [6]:
timestamps_df = pd.read_csv('timestamps.csv')

for _, row in timestamps_df.iterrows():
    print(f"Processing {row['sportmonks_id']}: {row['home_team']} vs {row['away_team']}")
    get_odds(row)
    time.sleep(5)

Processing 58328: Chennai Super Kings vs Royal Challengers Bangalore
Skipping 58328 : Chennai Super Kings vs Royal Challengers Bangalore - data exists
Processing 58331: Punjab Kings vs Delhi Capitals
Skipping 58331 : Punjab Kings vs Delhi Capitals - data exists
Processing 58334: Kolkata Knight Riders vs Sunrisers Hyderabad
Skipping 58334 : Kolkata Knight Riders vs Sunrisers Hyderabad - data exists
Processing 58337: Rajasthan Royals vs Lucknow Super Giants
Skipping 58337 : Rajasthan Royals vs Lucknow Super Giants - data exists
Processing 58340: Gujarat Titans vs Mumbai Indians
Saved odds for Gujarat Titans vs Mumbai Indians
Processing 58343: Royal Challengers Bangalore vs Punjab Kings
Saved odds for Royal Challengers Bangalore vs Punjab Kings
Processing 58346: Chennai Super Kings vs Gujarat Titans
404 Client Error: Not Found for url: https://api.the-odds-api.com/v4/historical/sports/cricket_ipl/events/fab1a85d378b7726d718c07b464a3f8e/odds?apiKey=913dc83e6ebd4214af163b1ca4cde116&regions=

KeyboardInterrupt: 